In [16]:
import pandas as pd 
import numpy as np 
import plotly.express as px

In [13]:
# https://www.cbs.nl/nl-nl/maatwerk/2024/35/kerncijfers-wijken-en-buurten-2024
df = pd.read_excel('./kwb-2024.xlsx')
df.shape

(18310, 131)

In [24]:
df.head()

,gwb_code_10,gwb_code_8,regio,gm_naam,recs,gwb_code,ind_wbi,a_inw,a_man,a_vrouw,...,a_lan_ha,a_wat_ha,pst_mvp,pst_dekp,ste_mvs,ste_oad,single_women,single_women_proportion,single_women_total,single_women_age_group
1,GM0014,0014,Groningen,Groningen,Gemeente,GM0014,1,243768,120912,122856,...,18553,1244,.,.,1,3472,4.493020e+09,82451.330314,1.012964e+10,2.264426e+09
2,WK001400,001400,Centrum,Groningen,Wijk,WK001400,1,24920,12845,12070,...,228,13,.,.,1,6766,1.162414e+08,10314.231541,1.244928e+08,5.630151e+07
3,BU00140000,00140000,Binnenstad-Noord,Groningen,Buurt,BU00140000,1,4745,2470,2270,...,37,2,9712,1,1,7001,4.740616e+06,2004.488936,4.550190e+06,2.267903e+06
4,BU00140001,00140001,Binnenstad-Zuid,Groningen,Buurt,BU00140001,1,6975,3640,3340,...,55,4,9711,1,1,6698,1.033465e+07,2940.157706,9.820127e+06,4.948781e+06
5,BU00140002,00140002,Binnenstad-Oost,Groningen,Buurt,BU00140002,1,4400,2275,2125,...,27,2,9711,3,1,6355,3.326555e+06,1832.812500,3.894727e+06,1.606575e+06


In [20]:
df = df[df['recs'] != "Land"]

In [39]:
def plot_best_chances_for_singles(df, col, age_classes, min_population=10000, min_percentage=0.1):
    """
    Plots the regions or municipalities where single men have the best chances 
    based on the proportion of single women and a minimum population threshold.

    Args:
        df (pd.DataFrame): Input DataFrame with columns for demographics.
        col (str): The column to group by, e.g., 'gm_naam' or 'regio'.
        age_classes (list): List of age class columns to analyze, e.g., ['a_25_44', 'a_15_24'].
        min_population (int): Minimum population size to include in the analysis.
        min_percentage (float): Minimum proportion of single women to include (e.g., 0.1 for 10%).
    """
    # Ensure valid rows
    df = df.dropna(subset=["a_ongeh", "a_vrouw", "a_inw"] + age_classes)
    df = df[df["a_inw"] > 0]  # Exclude rows with zero population to avoid division errors

    for age_class in age_classes:
        if age_class not in df.columns:
            print(f"Age class '{age_class}' is not in the DataFrame. Skipping...")
            continue
        
        # Step 1: Calculate unmarried women proportion
        df["unmarried_women"] = df["a_ongeh"] * (df["a_vrouw"] / df["a_inw"])
        
        # Step 2: Adjust for the age group
        df["unmarried_women_age_group"] = df["unmarried_women"] * (df[age_class] / df["a_inw"])
        
        # Step 3: Calculate proportion of unmarried women relative to total population
        df["unmarried_women_proportion"] = df["unmarried_women_age_group"] / df["a_inw"]
        
        # Step 4: Filter by minimum population size and unmarried women proportion
        df_filtered = df[(df["a_inw"] >= min_population) & 
                         (df["unmarried_women_proportion"] >= min_percentage)]
        
        # Step 5: Get the top 10 regions based on unmarried women proportion
        top_10 = df_filtered.nlargest(10, "unmarried_women_proportion")
        
        # Step 6: Plot the data
        fig = px.bar(
            top_10,
            x=col,
            y="unmarried_women_proportion",
            title=f"Top 10 {col} Where Single Men Have the Best Chances ({age_class})",
            labels={col: col.capitalize(), "unmarried_women_proportion": "Proportion of Single Women"},
            text="unmarried_women_proportion",
        )
        fig.update_traces(texttemplate="%{text:.2%}", textposition="outside")
        fig.update_layout(
            xaxis_title=col.capitalize(),
            yaxis_title="Proportion of Single Women",
            uniformtext_minsize=8,
            uniformtext_mode="hide"
        )
        fig.show()

# Example usage
plot_best_chances_for_singles(df, col="regio", age_classes=["a_25_44", "a_15_24"], min_population=10000, min_percentage=0.15)

In [40]:
df.columns

Index(['gwb_code_10', 'gwb_code_8', 'regio', 'gm_naam', 'recs', 'gwb_code',
       'ind_wbi', 'a_inw', 'a_man', 'a_vrouw',
       ...
       'pst_mvp', 'pst_dekp', 'ste_mvs', 'ste_oad', 'single_women',
       'single_women_proportion', 'single_women_total',
       'single_women_age_group', 'unmarried_women',
       'unmarried_women_age_group'],
      dtype='object', length=137)

In [41]:
inw = 12000
won = round(inw / 1.75)
print(inw, won)

12000 6857


In [42]:

df = df[df['a_inw'] >= inw]
df = df[df['a_woning'].astype(int) >= won]
df.shape

(548, 137)

In [43]:
df = df[['regio', 'gm_naam', 'recs', 'gwb_code']]

In [44]:
df['recs'].value_counts()

recs
Gemeente    304
Wijk        230
Buurt        14
Name: count, dtype: int64

In [45]:
df1 = df[(df['recs'] != "Land") & (df['recs'] != "Buurt")]
df1.shape

(534, 4)

In [46]:
df1.head(2)

,regio,gm_naam,recs,gwb_code
1,Groningen,Groningen,Gemeente,GM0014
2,Centrum,Groningen,Wijk,WK001400


In [47]:
regio_list = df1['regio'].to_list()

In [48]:
regio_list

['Groningen',
 'Centrum',
 'Oud-Zuid',
 'Oud-West',
 'Oud-Noord',
 'Oosterparkwijk',
 'Helpman e.o.',
 'Nieuw-West',
 'Noordwest',
 'Noordoost',
 'Noorddijk e.o.',
 'Almere',
 'Stadskanaal',
 'Stadskanaal',
 'Veendam',
 'Wijk 00 Veendam-kern',
 'Zeewolde',
 'Wijk 00',
 'Achtkarspelen',
 'Harlingen',
 'Wijk 00 Harlingen',
 'Heerenveen',
 'Wijk 01 Heerenveen',
 'Leeuwarden',
 'Oud-Oost',
 'Ooststellingwerf',
 'Opsterland',
 'Smallingerland',
 'Weststellingwerf',
 'Assen',
 'Coevorden',
 'Wijk 10 Coevorden',
 'Emmen',
 'Hoogeveen',
 'Wijk 50 Hoogeveen',
 'Meppel',
 'Almelo',
 'Borne',
 'Wijk 00 Borne',
 'Dalfsen',
 'Deventer',
 'Enschede',
 'Wijk 00 Binnensingelgebied',
 'Wijk 02 Boswinkel - Stadsveld',
 'Wijk 04 Enschede-Noord',
 'Wijk 06 Enschede-Zuid',
 'Wijk 08 Glanerbrug en omgeving',
 'Haaksbergen',
 'Wijk 00 Haaksbergen (dorp)',
 'Hardenberg',
 'Hardenberg',
 'Hellendoorn',
 'Hengelo (O.)',
 'Kampen',
 'Kampen',
 'Losser',
 'Noordoostpolder',
 'Wijk 01 Emmeloord',
 'Oldenzaal',
 'O

In [50]:
# Open a file for writing
with open('regio_list.txt', 'w') as file:
    # Format the list with 8 items per line
    formatted_list = ',\n'.join(
        ', '.join(f'"{item}"' for item in regio_list[i:i + 8])
        for i in range(0, len(regio_list), 8)
    )
    file.write(f"[{formatted_list}]\n")


In [ ]:
# rotate proxy
# robot.xtx
# make sure it has more delay


In [55]:
data = pd.read_csv("data.csv")
data.shape

(19960, 9)

In [56]:
data.columns

Index(['name', 'adres', 'website', 'telefoon', 'reviews_count',
       'reviews_average', 'latitude', 'longitude', ' search_keyword'],
      dtype='object')

In [57]:
data = data.sort_values(by='reviews_count', ascending=False)
data = data.drop_duplicates(subset=['name', 'website'], keep='first')

# Drop rows where 'website' is NaN or an empty string
data = data[data['website'].notna() & (data['website'] != "")]
data.shape

(3969, 9)

In [ ]:
# use data seen to not read df every loop to make it faster
# Scrape only data that has websites for faster handle 
# update script to be callable by class to run paralel without copy .py
# Modify the code so it can run multi browser and boost speed.
# Append keyword used into the csv for storage so we know where it came from
# maybe append takes much ram on csv